<img src="images/nvidia_header.png" style="margin-left: -30px; width: 300px; float: left;">

# NVIDIA 모델 압축 종합 실습

이 노트북은 모델 압축의 전체 워크플로우를 실습합니다.

## 목차
1. [환경 설정 및 기준 모델](#1.-환경-설정-및-기준-모델)
2. [양자화 (Quantization)](#2.-양자화-Quantization)
3. [프루닝 (Pruning)](#3.-프루닝-Pruning)
4. [지식 증류 (Knowledge Distillation)](#4.-지식-증류-Knowledge-Distillation)
5. [추론 배포 및 성능 측정](#5.-추론-배포-및-성능-측정)
6. [종합 비교 및 분석](#6.-종합-비교-및-분석)

## 1. 환경 설정 및 기준 모델

### 1.1 필요한 라이브러리 Import

In [ ]:
import os
import shutil
import time
import json
import logging
from typing import List, Dict

# NeMo & TensorRT
from nemo.export.tensorrt_llm import TensorRTLLM
from nemo.deploy import DeployPyTriton
from nemo.deploy.nlp import NemoQueryLLM

# 시각화
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display, Markdown

# 로깅 설정
logging.getLogger("TRT-LLM").setLevel(logging.ERROR)

print("✅ 라이브러리 import 완료")

### 1.2 전역 설정

In [ ]:
# 모델 경로
HF_CHECKPOINT = "Llama-3.2-3B"
NEMO_CHECKPOINT = "Llama-3.2-3B.nemo"

# 디렉토리 설정
os.makedirs("qnemo", exist_ok=True)
os.makedirs("engines", exist_ok=True)
os.makedirs("experiments", exist_ok=True)
os.makedirs("results", exist_ok=True)

# 엔진 구성
MAX_BS = 4
MAX_INPUT_LEN = 2048
MAX_OUTPUT_LEN = 2048

print("✅ 전역 설정 완료")

### 1.3 NeMo 체크포인트 준비

In [ ]:
# NeMo 체크포인트 이동
src_file = os.path.join("/dli/task/Llama-3.2-3B", "Llama-3.2-3B.nemo")
dst_file = os.path.join("/dli/task", "Llama-3.2-3B.nemo")

if os.path.exists(dst_file):
    print(f"✅ NeMo 체크포인트 준비 완료: {dst_file}")
else:
    print(f"⏳ {src_file} 다운로드 대기 중...")
    while not os.path.exists(src_file):
        time.sleep(10)
        print(f"여전히 대기 중...")
    
    shutil.move(src_file, dst_file)
    print(f"✅ 체크포인트 이동 완료: {dst_file}")

### 1.4 기준 모델 (BF16) 엔진 빌드

먼저 BF16 정밀도로 기준 모델의 TensorRT-LLM 엔진을 빌드합니다.

In [ ]:
%%bash
NEMO_CHECKPOINT="Llama-3.2-3B.nemo"
QNEMO_PATH="qnemo/llama-3.2-3b/bf16"

python3 /opt/NeMo/examples/nlp/language_modeling/megatron_gpt_ptq.py \
    model.restore_from_path=$NEMO_CHECKPOINT \
    trainer.precision=bf16 \
    quantization.algorithm=null \
    export.decoder_type=llama \
    export.inference_tensor_parallel=1 \
    export.save_path=$QNEMO_PATH

In [ ]:
%%bash
ENGINE_DIR="engines/llama-3.2-3b/bf16"
QNEMO_PATH="qnemo/llama-3.2-3b/bf16"

trtllm-build --checkpoint_dir $QNEMO_PATH \
    --gemm_plugin auto \
    --output_dir $ENGINE_DIR \
    --max_batch_size 4 \
    --max_input_len 2048 \
    --max_seq_len 2048 \
    --gather_context_logits

### 1.5 기준 모델 MMLU 평가

In [ ]:
%%bash
HF_CHECKPOINT="Llama-3.2-3B"
ENGINE_DIR="engines/llama-3.2-3b/bf16"

python3 /workspace/tensorrtllm_backend/tensorrt_llm/examples/mmlu.py \
    --data_dir "/workspace/data/mmlu" \
    --model_dir $HF_CHECKPOINT \
    --engine_dir $ENGINE_DIR \
    --test_trt_llm

**예상 결과:** MMLU 정확도 약 **75%**

이 값이 우리의 기준선이 됩니다.

---

## 2. 양자화 (Quantization)

### 2.1 양자화 개념

양자화는 모델의 가중치와 활성화를 더 낮은 비트 표현으로 변환하여:
- 모델 크기 감소
- 메모리 대역폭 감소
- 추론 속도 향상

이 섹션에서는 **FP8 양자화**를 적용합니다.

### 2.2 FP8 QNeMo 체크포인트 생성

FP8 양자화는 다음 단계로 진행됩니다:
1. 캘리브레이션 (활성화 평활화)
2. 가중치 양자화
3. QNeMo 체크포인트 저장

In [ ]:
%%bash
NEMO_CHECKPOINT="Llama-3.2-3B.nemo"
QNEMO_PATH="qnemo/llama-3.2-3b/fp8"

python3 /opt/NeMo/examples/nlp/language_modeling/megatron_gpt_ptq.py \
    model.restore_from_path=$NEMO_CHECKPOINT \
    trainer.precision=bf16 \
    quantization.algorithm=fp8 \
    export.decoder_type=llama \
    export.inference_tensor_parallel=1 \
    export.save_path=$QNEMO_PATH

**관찰 사항:**
- 로그에서 `Calibration` 항목을 확인하세요
- `TensorQuantizer(amax=[...])` - 양자화 범위
- 가중치와 활성화의 양자화 차이

### 2.3 FP8 TensorRT 엔진 빌드

In [ ]:
%%bash
ENGINE_DIR="engines/llama-3.2-3b/fp8"
QNEMO_PATH="qnemo/llama-3.2-3b/fp8"

trtllm-build --checkpoint_dir $QNEMO_PATH \
    --gemm_plugin auto \
    --output_dir $ENGINE_DIR \
    --max_batch_size 4 \
    --max_input_len 2048 \
    --max_seq_len 2048 \
    --gather_context_logits

### 2.4 양자화된 모델 테스트

In [ ]:
%%bash
HF_CHECKPOINT="Llama-3.2-3B"
ENGINE_DIR="engines/llama-3.2-3b/fp8"

python3 /workspace/tensorrtllm_backend/tensorrt_llm/examples/run.py \
    --engine_dir $ENGINE_DIR \
    --tokenizer_dir $HF_CHECKPOINT \
    --max_output_len 25 \
    --input_text "What is a graphics processing unit?" \
    --run_profiling

### 2.5 FP8 모델 MMLU 평가

In [ ]:
%%bash
HF_CHECKPOINT="Llama-3.2-3B"
ENGINE_DIR="engines/llama-3.2-3b/fp8"

python3 /workspace/tensorrtllm_backend/tensorrt_llm/examples/mmlu.py \
    --data_dir "/workspace/data/mmlu" \
    --model_dir $HF_CHECKPOINT \
    --engine_dir $ENGINE_DIR \
    --test_trt_llm

**예상 결과:** MMLU 정확도 약 **73%**

분석:
- 기준 대비 2%p 손실
- 모델 크기 50% 감소
- 추론 속도 약 2배 향상

---

## 3. 프루닝 (Pruning)

### 3.1 프루닝 개념

프루닝은 신경망에서 불필요한 부분을 제거합니다:
- **깊이 프루닝**: 전체 레이어 제거
- **폭 프루닝**: 개별 뉴런/헤드 제거

이 섹션에서는 **깊이 프루닝**을 수행하여 마지막 25% 레이어를 제거합니다.

### 3.2 깊이 프루닝 수행

Llama-3.2-3B는 28개의 레이어를 가지고 있습니다.
25% 제거 → 마지막 7개 레이어 (21-27) 삭제

In [ ]:
%%bash
python /opt/NeMo/examples/nlp/language_modeling/megatron_gpt_drop_layers.py \
    --path_to_nemo "Llama-3.2-3B.nemo" \
    --path_to_save "Llama-3.2-3B-pruned.nemo" \
    --tensor_model_parallel_size 1 \
    --pipeline_model_parallel_size 1 \
    --gpus_per_node 1 \
    --drop_layers 21 22 23 24 25 26 27

### 3.3 프루닝된 모델 평가

lm-evaluation-harness를 사용하여 평가합니다.

In [ ]:
%%bash
lm_eval --model nemo_lm \
    --model_args path="Llama-3.2-3B-pruned.nemo" \
    --tasks mmlu_high_school_world_history,mmlu_high_school_government_and_politics,mmlu_world_religions,mmlu_sociology \
    --num_fewshot 3 \
    --batch_size 4

**예상 결과:** MMLU 정확도 약 **65%**

분석:
- 기준 대비 10%p 손실
- 모델 크기 25% 감소
- ⚠️ 성능 저하 - 증류 필요!

---

## 4. 지식 증류 (Knowledge Distillation)

### 4.1 증류 개념

지식 증류는 큰 모델(TEACHER)의 지식을 작은 모델(STUDENT)에게 전달합니다:
- **TEACHER**: Llama-3.2-3B.nemo (원본)
- **STUDENT**: Llama-3.2-3B-pruned.nemo (프루닝됨)
- **데이터**: WikiText-103-v1

목표: 프루닝으로 손실된 성능 복구

### 4.2 TensorBoard 시작

증류 진행 상황을 모니터링하기 위해 TensorBoard를 시작합니다.

In [ ]:
%%js
const href = window.location.hostname;
let a = document.createElement("a");
a.textContent = "Open TensorBoard";
a.href = "http://" + href + ":8886";
a.style.cssText = "text-decoration: none; color: white; font-size: 20px; font-weight: bold; padding: 15px 25px; background-color: #76B900; border-radius: 8px; display: inline-block; box-shadow: 2px 2px 10px rgba(0, 0, 0, 0.2);";
a.target = "_blank";
a.onmouseover = () => a.style.backgroundColor = "#5a9300";
a.onmouseout = () => a.style.backgroundColor = "#76B900";
element.appendChild(a);

### 4.3 지식 증류 실행

⏱️ **예상 소요 시간**: 약 20-30분

In [ ]:
%%bash
export CUDA_DEVICE_MAX_CONNECTIONS=1

# 설정 변수
TENSOR_PARALLEL_SIZE=1
EXPERIMENT_DIR="./experiments/"
EXPERIMENT_NAME="llama_distill_depth_pruned_student"

# 모델 경로
TEACHER="Llama-3.2-3B.nemo"
STUDENT="Llama-3.2-3B-pruned.nemo"
FINAL_MODEL_PATH="Llama-3.2-3B-distilled.nemo"

# 데이터 경로
DATA_TRAIN='./wikidata/wikitext_tokenized_train_text_document'
DATA_VAL='./wikidata/wikitext_tokenized_val_text_document'
DATA_TEST='./wikidata/wikitext_tokenized_test_text_document'

# 토크나이저
TOK='Llama-3.2-3B'

# 하이퍼파라미터
STEPS=30
GLOBAL_BATCH_SIZE=64
MICRO_BATCH_SIZE=1
LR=7e-5
MIN_LR=1e-5
WARMUP_STEPS=2

# 증류 실행
torchrun --nproc-per-node=${TENSOR_PARALLEL_SIZE} \
    /opt/NeMo/examples/nlp/language_modeling/megatron_gpt_distillation.py \
    name=${EXPERIMENT_NAME} \
    exp_manager.exp_dir=${EXPERIMENT_DIR} \
    exp_manager.checkpoint_callback_params.save_top_k=1 \
    trainer.max_steps=${STEPS} \
    trainer.log_every_n_steps=1 \
    trainer.val_check_interval=50 \
    trainer.limit_val_batches=1 \
    trainer.precision=bf16 \
    trainer.devices=${TENSOR_PARALLEL_SIZE} \
    trainer.num_nodes=1 \
    "model.data.data_prefix={train:[1.0,$DATA_TRAIN],validation:[$DATA_VAL],test:[$DATA_TEST]}" \
    model.restore_from_path=${STUDENT} \
    model.kd_teacher_restore_from_path=${TEACHER} \
    model.nemo_path=${FINAL_MODEL_PATH} \
    model.tokenizer.type=${TOK} \
    model.tensor_model_parallel_size=${TENSOR_PARALLEL_SIZE} \
    model.micro_batch_size=${MICRO_BATCH_SIZE} \
    model.global_batch_size=${GLOBAL_BATCH_SIZE} \
    model.optim.lr=${LR} \
    model.optim.sched.min_lr=${MIN_LR} \
    model.optim.sched.warmup_steps=${WARMUP_STEPS} \
    +exp_manager.create_tensorboard_logger=True \
    model.data.seq_length=3072

**TensorBoard에서 확인할 사항:**
- 손실(Loss) 감소 추세
- KL Divergence 수렴
- 학습률(Learning Rate) 스케줄

### 4.4 증류된 모델 평가

In [ ]:
%%bash
lm_eval --model nemo_lm \
    --model_args path="./Llama-3.2-3B-distilled.nemo" \
    --tasks mmlu_high_school_world_history,mmlu_high_school_government_and_politics,mmlu_world_religions,mmlu_sociology \
    --num_fewshot 3 \
    --batch_size 4

**예상 결과:** MMLU 정확도 약 **72%**

분석:
- 프루닝 대비 7%p 향상!
- 기준 대비 3%p 손실만
- 크기는 75%로 유지
- ✅ 증류 성공!

---

## 5. 추론 배포 및 성능 측정

### 5.1 증류된 모델의 TensorRT 엔진 빌드

#### 5.1.1 QNeMo 체크포인트 생성

In [ ]:
%%bash
NEMO_CHECKPOINT="Llama-3.2-3B-distilled.nemo"
QNEMO_PATH="qnemo/llama-3.2-3b-distilled/bf16"

python3 /opt/NeMo/examples/nlp/language_modeling/megatron_gpt_ptq.py \
    model.restore_from_path=$NEMO_CHECKPOINT \
    trainer.precision=bf16 \
    quantization.algorithm=null \
    export.decoder_type=llama \
    export.inference_tensor_parallel=1 \
    export.save_path=$QNEMO_PATH

#### 5.1.2 TensorRT 엔진 빌드 (Python API)

In [ ]:
from nemo.export.tensorrt_llm import TensorRTLLM

trt_llm_exporter = TensorRTLLM(model_dir="engines/llama-3.2-3b-distilled/bf16")
trt_llm_exporter.export(
    nemo_checkpoint_path="qnemo/llama-3.2-3b-distilled/bf16",
    model_type="llama",
    tensor_parallelism_size=1,
)

# 테스트 추론
test_output = trt_llm_exporter.forward(["What is a graphics processing unit?"])
print("테스트 출력:", test_output)

### 5.2 Triton Inference Server 배포

In [ ]:
from nemo.deploy import DeployPyTriton

# Triton 서버 시작
nm = DeployPyTriton(
    model=trt_llm_exporter,
    triton_model_name="llama",
    http_port=8000
)

nm.deploy()
nm.run()

print("✅ Triton 서버 시작 완료: http://localhost:8000")

### 5.3 추론 테스트

In [ ]:
from nemo.deploy.nlp import NemoQueryLLM

nq = NemoQueryLLM(url="localhost", model_name="llama")
prompts = ["What is a graphics processing unit?"]
output = nq.query_llm(prompts=prompts, max_output_len=50)

print("프롬프트:", prompts[0])
print("\n출력:", output[0])

### 5.4 성능 측정

#### 지연 시간 (Latency) 및 처리량 (Throughput) 측정

In [ ]:
import time

# 설정
num_requests = 5
batch_size = 4
input_length = 400
output_length = 100

# 테스트 프롬프트 생성
prompts = ["0" * input_length] * batch_size

# 성능 측정
start_time = time.time()
for _ in range(num_requests):
    output = nq.query_llm(prompts=prompts, max_output_len=output_length)
end_time = time.time()

# 계산
total_time = end_time - start_time
throughput = num_requests * batch_size / total_time
latency = total_time / num_requests

print(f"🚀 처리량: {throughput:.2f} requests/second")
print(f"⏱️  평균 지연 시간: {latency:.2f} seconds")
print(f"📊 총 요청 수: {num_requests * batch_size}")
print(f"⏰ 총 소요 시간: {total_time:.2f} seconds")

### 5.5 Triton 서버 중지

다른 모델을 테스트하기 전에 서버를 중지합니다.

In [ ]:
# 서버 중지 (필요한 경우)
nm.stop()
print("✅ Triton 서버 중지 완료")

---

## 6. 종합 비교 및 분석

### 6.1 성능 결과 정리

In [ ]:
# 결과 데이터 (실제 측정값으로 업데이트)
results = {
    '모델': [
        'BF16 기준',
        'FP8 양자화',
        'BF16 프루닝',
        'BF16 증류',
        'FP8 증류'
    ],
    'MMLU (%)': [75, 73, 65, 72, 70],
    '상대 크기 (%)': [100, 50, 75, 75, 37.5],
    '처리량 (req/s)': [10, 18, 13, 13, 24],
    '메모리 (GB)': [6.4, 3.2, 4.8, 4.8, 2.4]
}

df = pd.DataFrame(results)
display(df)

# CSV로 저장
df.to_csv('results/compression_comparison.csv', index=False)
print("\n✅ 결과를 results/compression_comparison.csv에 저장했습니다.")

### 6.2 시각화

#### 6.2.1 MMLU 정확도 비교

In [ ]:
fig, ax = plt.subplots(figsize=(12, 6))

colors = ['#1f77b4', '#76b900', '#ff7f0e', '#2ca02c', '#d62728']
bars = ax.bar(df['모델'], df['MMLU (%)'], color=colors)

# 값 표시
for bar in bars:
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2., height,
            f'{height:.0f}%',
            ha='center', va='bottom', fontsize=12, fontweight='bold')

ax.set_ylabel('MMLU 정확도 (%)', fontsize=14, fontweight='bold')
ax.set_title('모델 압축 기술별 MMLU 정확도 비교', fontsize=16, fontweight='bold')
ax.set_ylim(0, 100)
ax.grid(axis='y', alpha=0.3)
plt.xticks(rotation=15, ha='right')
plt.tight_layout()
plt.savefig('results/mmlu_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

print("✅ 그래프를 results/mmlu_comparison.png에 저장했습니다.")

#### 6.2.2 파레토 곡선 (정확도 vs 처리량)

In [ ]:
fig, ax = plt.subplots(figsize=(12, 8))

# 산점도
scatter = ax.scatter(df['처리량 (req/s)'], df['MMLU (%)'], 
                     s=300, c=colors, alpha=0.6, edgecolors='black', linewidth=2)

# 레이블 추가
for i, model in enumerate(df['모델']):
    ax.annotate(model, 
                (df['처리량 (req/s)'][i], df['MMLU (%)'][i]),
                textcoords="offset points",
                xytext=(0, 10),
                ha='center',
                fontsize=10,
                fontweight='bold')

# 파레토 프론티어 표시
pareto_indices = [0, 1, 3, 4]  # BF16 기준, FP8, BF16 증류, FP8 증류
pareto_df = df.iloc[pareto_indices].sort_values('처리량 (req/s)')
ax.plot(pareto_df['처리량 (req/s)'], pareto_df['MMLU (%)'], 
        'r--', alpha=0.5, linewidth=2, label='파레토 프론티어')

ax.set_xlabel('처리량 (requests/second)', fontsize=14, fontweight='bold')
ax.set_ylabel('MMLU 정확도 (%)', fontsize=14, fontweight='bold')
ax.set_title('모델 압축: 정확도 vs 처리량 (파레토 곡선)', fontsize=16, fontweight='bold')
ax.grid(True, alpha=0.3)
ax.legend(fontsize=12)
plt.tight_layout()
plt.savefig('results/pareto_curve.png', dpi=300, bbox_inches='tight')
plt.show()

print("✅ 그래프를 results/pareto_curve.png에 저장했습니다.")

#### 6.2.3 메모리 사용량 비교

In [ ]:
fig, ax = plt.subplots(figsize=(12, 6))

bars = ax.barh(df['모델'], df['메모리 (GB)'], color=colors)

# 값 표시
for bar in bars:
    width = bar.get_width()
    ax.text(width, bar.get_y() + bar.get_height()/2.,
            f'{width:.1f} GB',
            ha='left', va='center', fontsize=12, fontweight='bold')

ax.set_xlabel('GPU 메모리 사용량 (GB)', fontsize=14, fontweight='bold')
ax.set_title('모델 압축 기술별 메모리 사용량 비교', fontsize=16, fontweight='bold')
ax.grid(axis='x', alpha=0.3)
plt.tight_layout()
plt.savefig('results/memory_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

print("✅ 그래프를 results/memory_comparison.png에 저장했습니다.")

### 6.3 압축률 분석

In [ ]:
# 압축률 계산
df['압축률'] = df['상대 크기 (%)'].apply(lambda x: f"{100 - x:.1f}%")
df['정확도 손실'] = df['MMLU (%)'].apply(lambda x: f"{75 - x:.1f}%p")
df['속도 향상'] = df['처리량 (req/s)'].apply(lambda x: f"{x / 10:.2f}x")

summary_df = df[['모델', '압축률', '정확도 손실', '속도 향상']]
display(summary_df)

print("\n📊 주요 발견:")
print("1. FP8 양자화: 50% 압축, 2%p 손실, 1.8배 속도 향상")
print("2. 프루닝: 25% 압축, 10%p 손실 (증류 전)")
print("3. 증류: 프루닝 손실의 70% 복구")
print("4. FP8 + 증류: 62.5% 압축, 5%p 손실, 2.4배 속도 향상")
print("\n✨ 최적 조합: FP8 양자화 + 증류 = 최고의 압축률과 성능 균형")

### 6.4 사용 사례별 권장사항

In [ ]:
recommendations = {
    '사용 사례': [
        '클라우드 배포',
        '엣지 장치',
        '실시간 서비스',
        '배치 처리',
        '모바일'
    ],
    '권장 기술': [
        'FP8 양자화',
        'INT4 + 프루닝',
        'FP8',
        'BF16 프루닝',
        'INT4 + 증류'
    ],
    '이유': [
        '비용 절감, 높은 처리량',
        '메모리 제약, 경량화',
        '낮은 지연, 좋은 품질',
        '정확도 우선',
        '극도의 경량화'
    ]
}

rec_df = pd.DataFrame(recommendations)
display(rec_df)

---

## 마무리

### 학습 내용 정리

이 노트북을 통해 다음을 학습했습니다:

1. **양자화 (Quantization)**
   - FP8 양자화 적용
   - 50% 크기 감소, 2%p 정확도 손실
   - TensorRT-LLM 사용

2. **프루닝 (Pruning)**
   - 깊이 프루닝으로 25% 레이어 제거
   - 초기 10%p 정확도 손실
   - NeMo Framework 사용

3. **지식 증류 (Knowledge Distillation)**
   - 원본 모델에서 프루닝 모델로 지식 전이
   - 7%p 정확도 복구
   - KL Divergence 최소화

4. **추론 배포**
   - Triton Inference Server 사용
   - 지연 시간 및 처리량 측정
   - 성능 벤치마킹

5. **종합 분석**
   - 파레토 곡선 분석
   - 사용 사례별 최적 기술 선택
   - 압축률 vs 정확도 트레이드오프
---

<center><a href="https://www.nvidia.com/en-us/training/"><img src="https://dli-lms.s3.amazonaws.com/assets/general/DLI_Header_White.png" width="400" height="186" /></a></center>